<a href="https://colab.research.google.com/github/rodrigorenemenegazzo/Python_QuantumComputing/blob/main/Implementando_algoritmo_QAOA_em_Qiskit_para_minimizar_fun%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Implementando algoritmo QAOA em Qiskit para minimizar função $$L(x_0, x_1, x_2) = -2x_0 + 3x_1 - 2x_2 -x_0 x_2 + 2 x_0 x_1$$

In [9]:
#Teste manual

X0 = 1
X1 = 0
X2 = 1

L = -2*X0+3*X1-2*X2-X0*X2+2*X0*X1
print(L)

-5


In [10]:
!pip install qiskit qiskit_algorithms qiskit_optimization


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=76ea4d5bde295e1cba27ec224ccc51d6a3eb50ae1eecfaa88c232d29861450ae
  Stored in directory: /root/.cache/pip/wheels/c1/d9/5c/1d919b1e441ebbaff244513a13d09c3c0378401601d4184bb0
Successfully built docplex


In [11]:
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.quantum_info import SparsePauliOp


##Criando instância da classe quadraticProgram -Definição das variaves do problema

In [13]:
problem = QuadraticProgram()

problem.binary_var('x0')
problem.binary_var('x1')
problem.binary_var('x2')


<Variable: x2 (binary)>

## Criando Termos da função de custo

In [14]:
linear_terms = {
    'x0': -2,
    'x1': 3,
    'x2': -2
}

## Criando Termos quadraticos. Valores para cada par de variáveis. No caso da x1 e x2 não temos então é 0.

In [15]:
quadratic_terms = {
    ('x0', 'x2'): -1,
    ('x0', 'x1'): 2,
    ('x1', 'x2'): 0
}


##Definindo a função de custo

In [18]:
problem.minimize(
    linear=linear_terms,
    quadratic=quadratic_terms
)
print(problem.prettyprint())


Problem name: 

Minimize
  2*x0*x1 - x0*x2 - 2*x0 + 3*x1 - 2*x2

Subject to
  No constraints

  Binary variables (3)
    x0 x1 x2



##Instanciar os otimizadores. Otimizador clássico - 100 iterações (pouco), para problemas maiores usar bem mais, demorará maistempo também.

In [19]:
optimizer = COBYLA(100)
sampler = StatevectorSampler()
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)

#Ajustes dos parametros
algorithm = MinimumEigenOptimizer(qaoa)
result = algorithm.solve(problem)

/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


In [20]:
#Visualização da solução (Melhor solução, sequencia de valores x0, x1, x2)
print('Melhor solução (x0, x1, x2): ', result.x)
print('Valor da função de custo: ', result.fval)
print('Status: ', result.status)

Melhor solução (x0, x1, x2):  [1. 0. 1.]
Valor da função de custo:  -5.0
Status:  OptimizationResultStatus.SUCCESS


## Precisa colocar restrições lineares para o TSP